<a href="https://colab.research.google.com/github/maxreynolds/Pattern-Cloud/blob/main/StreamlitFinetuningApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Setup code
!cd /content/
!pip install -q streamlit
!git clone https://github.com/huggingface/diffusers.git
!pip install -q ./diffusers
!pip install -q  -U -r /content/diffusers/examples/text_to_image/requirements.txt
!pip install -q wandb
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00
Cloning into 'diffusers'...
remote: Enumerating objects: 45475, done.
remote: Counting objects: 100% (2434/2434), done.
remote: Compressing objects: 100% (755/755), done.
remote: Total 45475 (delta 1732), reused 2082 (delta 1452), pack-reused 43041
Receiving objects: 100% (45475/45475), 28.84 MiB | 26.51 MiB/s, done.
Resolving deltas: 100% (33611/33611), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [3]:
!accelerate config default --mixed_precision fp16

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
import wandb
wandb.login()

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
#Making directories for multipage Streamlit applicaiton files and storing user images
!mkdir /content/pages
!mkdir /content/user_images
!ls

pages  sample_data  user_images


In [5]:
%%writefile /content/Upload.py
from diffusers import StableDiffusionPipeline
from datasets import load_dataset

import streamlit as st
import torch

from PIL import Image
import csv
import os

def delete_files_in_directory(directory_path):
   try:
     files = os.listdir(directory_path)
     for file in files:
       file_path = os.path.join(directory_path, file)
       if os.path.isfile(file_path):
         os.remove(file_path)
   except OSError:
     print("Error occurred while deleting files.")

image_set = []
image_set = st.file_uploader("Upload a folder of your training dataset Images:",
    type=["png"], accept_multiple_files=True) # TODO: expand to type=["png","jpg","jpeg"]]... need to update metadata.csv writer code to check for filetype

if image_set is not None:
    #st.write('Scroll down for next steps')
    st.image(image_set, width=128)

if st.button('Upload these Images as your training dataset', type="primary", use_container_width=True):
    placeholder = st.empty()
    placeholder.text("Formatting images...")
    for idx, image in enumerate(image_set):
      im = Image.open(image)
      im.save(f"/content/user_images/image{idx}.png")
    num_images = len(image_set)
    my_label = "<r4nd0m-l4b3l>"
    with open('/content/user_images/metadata.csv', 'w', newline='') as csvfile:
      spamwriter = csv.writer(csvfile, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
      spamwriter.writerow(['file_name,text'])
      for j in range(num_images):
        spamwriter.writerow([f'image{j}.png,'] + [my_label])
    placeholder.text('Loading dataset...')
    dataset = load_dataset("imagefolder", data_dir="/content/user_images", drop_labels=True)
    placeholder.text('Pushing to Hub...')
    dataset.push_to_hub("MaxReynolds/MyPatternDataset", private=False)
    placeholder.text('Upload complete!')
    #removing image files here
    delete_files_in_directory('/content/user_images')


Writing /content/Upload.py


In [4]:
%%writefile /content/pages/1Train.py
import streamlit as st
import subprocess

# should use this function at some point to remove pre-existing model from directory if it exists
def delete_files_in_directory(directory_path):
   try:
     files = os.listdir(directory_path)
     for file in files:
       file_path = os.path.join(directory_path, file)
       if os.path.isfile(file_path):
         os.remove(file_path)
   except OSError:
     print("Error occurred while deleting files.")

num_train_steps = st.slider('Select the number of training steps for your training process:', 0, 450, 250)

if st.button('Start Training on your Dataset', type="primary", use_container_width=True):
  with st.spinner('Training...'):
    subprocess.run(["accelerate", "launch", "diffusers/examples/text_to_image/train_text_to_image.py",
      "--pretrained_model_name_or_path=CompVis/stable-diffusion-v1-4",
      "--dataset_name=MaxReynolds/MyPatternDataset",
      "--use_ema",
      "--resolution=512",
      "--center_crop",
      "--random_flip",
      "--train_batch_size=1",
      "--gradient_accumulation_steps=4",
      "--gradient_checkpointing",
      f"--max_train_steps={num_train_steps}",
      "--learning_rate=1e-05",
      "--max_grad_norm=1",
      "--checkpointing_steps=100000",
      "--lr_scheduler=constant",
      "--lr_warmup_steps=0",
      "--push_to_hub",
      "--output_dir=MaxReynolds/MyPatternModel", #Can probably come up for a better name for this directory if not pushing to HF Hub (aka /content/models/)
      "--validation_prompt=,<r4nd0m-l4b3l>",
      "--report_to=wandb",
      "--seed=1337"])
  st.success('Training complete!')



Writing /content/pages/1Train.py


In [5]:
%%writefile /content/pages/2Generate.py
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image

import streamlit as st

if st.button('Generate some Images', type="primary", use_container_width=True):
  with st.spinner('Wait for it...'):
    model_path = "MaxReynolds/SouderRocketLauncherNetCombined"
    pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
    pipe.to("cuda")
    my_prompt = "Rocket Launcher by Lee Souder"
    num_images = 8
    # Run inference using above prompt to acquire some number of images
    all_images = [pipe(prompt=my_prompt).images[0] for i in range(num_images)]
    col1, col2, col3, col4 = st.columns(4)
    with col1:
      st.image(all_images[0])
      st.image(all_images[4])
    with col2:
      st.image(all_images[1])
      st.image(all_images[5])
    with col3:
      st.image(all_images[2])
      st.image(all_images[6])
    with col4:
      st.image(all_images[3])
      st.image(all_images[7])
  st.success('Done!')


Writing /content/pages/2Generate.py


In [8]:
!streamlit run /content/Upload.py &>/content/logs.txt &

In [9]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

Your tunnel url https://radiation-lives-webmaster-criterion.trycloudflare.com
